In [1]:
import pandas as pd
import nltk
import patsy
import statsmodels.api as sm
import seaborn as sn
import pymc3 as pm
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv', header = 0)

In [3]:
scores = data.points.copy()

In [5]:
model = sm.OLS(data.points[~(data.price.isnull()) & ~(data.points.isnull())], data.price[~(data.price.isnull()) & ~(data.points.isnull())])

In [6]:
results= model.fit()

In [7]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 points   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                 6.973e+04
Date:                Tue, 28 Aug 2018   Prob (F-statistic):               0.00
Time:                        20:47:40   Log-Likelihood:            -5.0963e+05
No. Observations:               90736   AIC:                         1.019e+06
Df Residuals:                   90735   BIC:                         1.019e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
price          1.0707      0.004    264.056      0.000       1.063       1.079
==============================================================================
Omnibus:                   204977.897   Durbin-Watson:                   0.843
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       4313220034.057
Skew:                         -21.039   Prob(JB):                         0.00
Kurtosis:                    1070.282   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
data_trimmed = data.drop(['description', 'Unnamed: 0', 'Unnamed: 0.1', 'title'], axis = 1)
data_trimmed.columns

Index(['country', 'designation', 'points', 'price', 'province', 'region_1',
       'region_2', 'taster_name', 'taster_twitter_handle', 'variety',
       'winery'],
      dtype='object')

In [9]:
dt= data_trimmed[['points', 'price', 'country', 'winery', 'variety', 'province']]

In [10]:
dt.head()

,points,price,country,winery,variety,province
0,92,22.0,US,Syncline,Rhône-style Red Blend,Washington
1,90,19.0,France,Château Laulerie,Bordeaux-style Red Blend,Southwest France
2,81,36.0,Argentina,Luigi Bosca,White Blend,Mendoza Province
3,86,24.0,France,Robert Karcher et Fils,Pinot Noir,Alsace
4,87,25.0,France,Maison Champy,Chardonnay,Burgundy


In [15]:
dt.isnull().sum()

points         0
price       6742
country       42
winery         0
variety        1
province      42
dtype: int64

In [27]:
country_codes = {v:k for k,v in enumerate(dt.country.unique())}

In [16]:
sample_data = pd.read_csv(pm.get_data('radon.csv'))

In [18]:
sample_data.head(200)

,Unnamed: 0,idnum,state,state2,stfips,zip,region,typebldg,floor,room,...,pcterr,adjwt,dupflag,zipflag,cntyfips,county,fips,Uppm,county_code,log_radon
0,0,5081.0,MN,MN,27.0,55735,5.0,1.0,1.0,3.0,...,9.7,1146.499190,1.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.832909
1,1,5082.0,MN,MN,27.0,55748,5.0,1.0,0.0,4.0,...,14.5,471.366223,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.832909
2,2,5083.0,MN,MN,27.0,55748,5.0,1.0,0.0,4.0,...,9.6,433.316718,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,1.098612
3,3,5084.0,MN,MN,27.0,56469,5.0,1.0,0.0,4.0,...,24.3,461.623670,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.095310
4,4,5085.0,MN,MN,27.0,55011,3.0,1.0,0.0,4.0,...,13.8,433.316718,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,1.163151
5,5,5086.0,MN,MN,27.0,55014,3.0,1.0,0.0,4.0,...,12.8,471.366223,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,0.955511
6,6,5087.0,MN,MN,27.0,55014,3.0,1.0,0.0,4.0,...,14.5,471.366223,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,0.470004
7,7,5088.0,MN,MN,27.0,55014,3.0,1.0,0.0,2.0,...,18.0,433.316718,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,0.095310
8,8,5089.0,MN,MN,27.0,55014,3.0,1.0,0.0,4.0,...,25.1,433.316718,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,-0.223144
9,9,5090.0,MN,MN,27.0,55014,3.0,1.0,0.0,4.0,...,17.5,461.623670,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,0.262364


In [37]:
start = dt[['points', 'price', 'country']]
start = pd.get_dummies(start)
droppedna = start.dropna()
droppedna.head()

,points,price,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Bosnia and Herzegovina,country_Brazil,country_Bulgaria,country_Canada,...,country_Serbia,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Switzerland,country_Turkey,country_US,country_Ukraine,country_Uruguay
0,92,22.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,90,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,81,36.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,86,24.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,87,25.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
model = sm.OLS(droppedna.points, droppedna.loc[:, 'price':])
results = model.fit()

In [39]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 points   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.539e+06
Date:                Tue, 28 Aug 2018   Prob (F-statistic):               0.00
Time:                        20:59:02   Log-Likelihood:            -2.3635e+05
No. Observations:               90736   AIC:                         4.728e+05
Df Residuals:                   90693   BIC:                         4.732e+05
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
price                              0.0294      0.000    111.217      0.000       0.029       0.030
country_Argentina                 85.9997      0.062   1394.741      0.000      85.879      86.121
country_Armenia                   87.0732      2.315     37.609      0.000      82.535      91.611
country_Australia                 87.5299      0.079   1106.114      0.000      87.375      87.685
country_Austria                   89.2776      0.072   1244.033      0.000      89.137      89.418
country_Bosnia and Herzegovina    86.1321      2.315     37.203      0.000      81.594      90.670
country_Brazil                    84.1639      0.524    160.516      0.000      83.136      85.192
country_Bulgaria                  87.5920      0.326    268.837      0.000      86.953      88.231
country_Canada                    88.3160      0.236    374.420      0.000      87.854      88.778
country_Chile                     85.9154      0.057   1498.259      0.000      85.803      86.028
country_China                     88.4702      3.274     27.020      0.000      82.053      94.888
country_Croatia                   86.6635      0.434    199.808      0.000      85.813      87.514
country_Cyprus                    86.6845      1.238     70.046      0.000      84.259      89.110
country_Czech Republic            86.5619      1.158     74.775      0.000      84.293      88.831
country_Egypt                   6.371e-13    4.2e-14     15.166      0.000    5.55e-13    7.19e-13
country_England                   89.9521      0.459    196.104      0.000      89.053      90.851
country_France                    87.5241      0.030   2880.479      0.000      87.465      87.584
country_Georgia                   86.8125      0.430    201.911      0.000      85.970      87.655
country_Germany                   88.6055      0.083   1070.777      0.000      88.443      88.768
country_Greece                    86.6391      0.172    502.460      0.000      86.301      86.977
country_Hungary                   88.0338      0.312    281.801      0.000      87.421      88.646
country_India                     89.4787      1.238     72.304      0.000      87.053      91.904
country_Israel                    87.5991      0.176    497.062      0.000      87.254      87.944
country_Italy                     87.4535      0.031   2830.364      0.000      87.393      87.514
country_Lebanon                   86.4356      0.668    129.319      0.000      85.126      87.746
country_Luxembourg                88.1525      1.464     60.202      0.000      85.282      91.022
country_Macedonia                 86.3011      1.158     74.551      0.000      84.032      88.570
country_Mexico                    84.2694      0.463    181.970      0.000      83.362      85.177
country_Moldov